In [4]:
import numpy as np # Pour la manipulation de tableaux
from keras.applications import  resnet50
from sklearn import metrics 
import os

import pandas as pd # Pour manipuler des DataFrames pandas

import matplotlib.pyplot as plt # Pour l'affichage d'images
from matplotlib import cm # Pour importer de nouvelles cartes de couleur
%matplotlib inline

from keras.models import Sequential # Pour construire un réseau de neurones
from keras.layers import Dense, Conv2D # Pour instancier une couche dense
from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Activation,GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.models import Model
import cv2
import keras
from numpy.random import seed
import tensorflow

from keras.preprocessing.image import ImageDataGenerator
import itertools # Pour créer des iterateurs

from sklearn import linear_model, preprocessing 



%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import time, cv2
import seaborn as sns

import pandas as pd
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
import pandas as pd


In [2]:
#Paramètres
base_img_paths="/home/marcpozzo/Desktop/c3po/Images_aquises/"
generateur_path='/mnt/VegaSlowDataDisk/c3po/Images_aquises/generateur_bigger.csv'
#generateur_path='/mnt/VegaSlowDataDisk/c3po/Chaine_de_traitement/Train_imagettes_annotées/type_oiseau/Materiel/generateur.csv'
test_size=0.1
epochs=200
batch_size = 600
zoom_range = 1.25
horizontal_flip = True
Minimum_Number_Class=100
dropout_rate=0.1
#steps_per_epoch=len(data_train)//batch_size
steps_per_epoch=1
#validation_steps=len(data_test)//batch_size
validation_steps=1


In [19]:
Images

,filename,classe,xmin,ymin,xmax,ymax,imagetteName
0,image_2019-06-14_15-46-54.jpg,corneille,709,305,722,317,image_2019-06-14_15-46-54_corneille_724.JPG
1,image_2019-06-14_15-46-54.jpg,ground,694,306,706,319,image_2019-06-14_15-46-54_ground_725.JPG
2,image_2019-06-14_15-46-54.jpg,corneille,762,313,774,324,image_2019-06-14_15-46-54_corneille_726.JPG
3,image_2019-06-14_15-46-54.jpg,ground,778,311,790,322,image_2019-06-14_15-46-54_ground_727.JPG
4,image_2019-06-14_15-47-11.jpg,corneille,755,293,774,308,image_2019-06-14_15-47-11_corneille_728.JPG
...,...,...,...,...,...,...,...
4513,image_2019-04-30_18-55-20.jpg,ground,393,494,403,506,image_2019-04-30_18-55-20_ground_6215.JPG
4514,image_2019-04-30_18-55-20.jpg,pigeon,625,423,633,431,image_2019-04-30_18-55-20_pigeon_6216.JPG
4515,image_2019-04-30_18-55-20.jpg,ground,615,421,623,429,image_2019-04-30_18-55-20_ground_6217.JPG
4516,image_2019-04-30_18-55-20.jpg,pigeon,1119,399,1140,412,image_2019-04-30_18-55-20_pigeon_6218.JPG


In [21]:

image_path="../../Materiels/images.csv"
Images=pd.read_csv(image_path)
Images["classe"].unique()


array(['corneille', 'ground', 'faisan', 'autre', 'ciel', 'lapin',
       'chevreuil', 'pigeon', 'incertain', 'cheval'], dtype=object)

In [ ]:
Minimum_Number_Class=100
def eliminate_small_categories(df,Minimum_Number_Class):
    numerous_labels_=[]
    all_labels_=df["classe"].unique()
    print("This is the list of different labels (acccording to the DataFrame) :",df["classe"].unique())
    print("Images are now deleting from data base if the population are below ",Minimum_Number_Class)  
    for i in df["classe"].unique():
        if df["classe"][df["classe"]==i].count()>Minimum_Number_Class:
            numerous_labels_.append(i)

    less_numerous_labels_=set(all_labels_)-set(Utilisable)
    #print(Non_Utilisable,"Non_Utilisable")
    for i in less_numerous_labels_:
        df=df[df["classe"]!=i] 

    print("This is the list of labels keep:", df["classe"].unique())
    return df
    
df=eliminate_small_categories(Images,Minimum_Number_Class)

In [27]:
#seed(1)
#tensorflow.random.set_seed(2)
test_size=0.2
data_train,data_test= train_test_split(df,stratify=df["classe"], test_size=test_size,random_state=42)

In [28]:
zoom_range=1.1
horizontal_flip=False
batch_size=16
train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        # data augmentation
        rotation_range = 10,
        zoom_range = zoom_range,
        horizontal_flip = horizontal_flip)

In [29]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)




train_generator = train_data_generator.flow_from_dataframe(dataframe=data_train,
                                                          directory="../../../../Pic_dataset",
                                                           x_col = "filename",
                                                           y_col = "classe",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = batch_size)


test_generator = test_data_generator.flow_from_dataframe(dataframe=data_test,
                                                          directory="../../../../Pic_dataset",
                                                           x_col = "filename",
                                                            y_col = "classe",
                                                           class_mode ="sparse",
                                                          target_size = (28 , 28), 
                                                          batch_size = batch_size)
NB_CLASSES=len(set(train_generator.classes))

Found 3594 validated image filenames belonging to 6 classes.
Found 899 validated image filenames belonging to 6 classes.


In [30]:
def Lenet_archi(NB_CLASSES,dropout_rate):
    lenet = Sequential()

    conv_1 = Conv2D(filters = 30,                     # Nombre de filtres
                    kernel_size = (5, 5),            # Dimensions du noyau
                    padding = 'valid',               # Mode de Dépassement
                    input_shape = (28, 28, 3),       # Dimensions de l'image en entrée
                    activation = 'relu')             # Fonction d'activation

    max_pool_1 = MaxPooling2D(pool_size = (2, 2))

    conv_2 = Conv2D(filters = 16,                    
                    kernel_size = (3, 3),          
                    padding = 'valid',             
                    activation = 'relu')

    max_pool_2 = MaxPooling2D(pool_size = (2, 2))

    flatten = Flatten()

    dropout = Dropout(rate = dropout_rate)

    dense_1 = Dense(units = 128,
                    activation = 'relu')

    dense_2 = Dense(units = 6,
                    activation = 'softmax')

    lenet.add(conv_1)
    lenet.add(max_pool_1)
    lenet.add(conv_2)
    lenet.add(max_pool_2)
    lenet.add(dropout)
    lenet.add(flatten)
    lenet.add(dense_1)
    lenet.add(dense_2)
    
    return lenet

In [31]:
dropout_rate=0.5
nn=Lenet_archi(NB_CLASSES,dropout_rate)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [32]:
# Compilation
nn.compile(loss='sparse_categorical_crossentropy',  # fonction de perte
              optimizer='adam',                 # algorithme de descente de gradient
              metrics=['accuracy'])             # métrique d'évaluation

In [34]:
epochs=2
history=nn.fit_generator( train_generator,
                           steps_per_epoch=len(data_train)//batch_size,
                           epochs=epochs,
                           workers=-1,
                           validation_data=test_generator,
                           validation_steps=len(data_test)//batch_size)
                          


plt.figure(figsize=(16,12))

plt.plot(history.history['accuracy'],label="train");
plt.plot(history.history['val_accuracy'],label="test");
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Accuracy en fonction du nombre d'epoch");
plt.legend();
#drop_out.50

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
224/224 [==============================] - 167s 747ms/step - loss: 8.3032 - acc: 0.4788 - val_loss: 8.0231 - val_acc: 0.5022
Epoch 2/2
224/224 [==============================] - 153s 685ms/step - loss: 7.9790 - acc: 0.5050 - val_loss: 8.0499 - val_acc: 0.5006


KeyError: 'accuracy'

<Figure size 1152x864 with 0 Axes>